# Importation des packages

In [1]:
from time import time
import numpy as np
import pandas as pd
import tensorflow as tf

# Import Regex to clean up tweets
import re

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

# Get Reviews
import requests
import json

# Get Tweets
import httplib2
import requests
import urllib3

# TF IDF Imports

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split  
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from joblib import dump, load

# RNN & LSTM Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression



In [2]:
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
data.head()
data = data.sort_values(by ='toxic', ascending = 0)
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
83844,e061b485cfa88d4e,Vandalism of my user page \n\nPlease do not va...,1,0,1,0,1,0
145225,190748c37c8dc159,"Douche bag, know the band Sublime before you t...",1,0,1,0,1,0
54826,9284484fe7d8fdd9,Interesting fact \n\nYou're a huge fucking too...,1,0,1,0,1,0
35554,5f0120e927cf78d9,"""::What point do you think you have? There's w...",1,0,0,0,0,0
145216,18f35bf40d79d277,FUCK YOU!!!! stupid IGNORANT!!!!! \n\nHIM\nis ...,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...
55891,955748da78822b3a,History of electric trams |from_oldid= http://...,0,0,0,0,0,0
55892,9557d6b1fde1ad09,"""\n\nFinished merge, it's now part of a new su...",0,0,0,0,0,0
55893,9557f142238078d1,""":::I misunderstood your reference to the arti...",0,0,0,0,0,0
55894,9558ca2330e0133a,"""\n\nI am going to sleep. I will think about t...",0,0,0,0,0,0


# Etude du jeu de données

In [5]:
# Your Code

# Préparation des données

In [6]:
# Your Code
labels = data.drop(['id','comment_text'],axis=1)
texts = data['comment_text']

stopwords_ = nltk.corpus.stopwords.words('english')
stopwords_[:10]




['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [7]:
stemmer = SnowballStemmer(language='english')
def simple_clear(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]", ' ', text)
    text = re.sub('<.*?>','',text)
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)
    text = re.sub('\[|\(.*\]|\)','', text) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)  
    text = text.split()
    text = [stemmer.stem(word) for word in text if not word in set(stopwords_)]
    text = ' '.join(text)
    return text

In [8]:
texts['clean'] = texts.apply(simple_clear)
texts['clean'][:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


83844     vandal user page pleas vandal user page also b...
145225    douch bag know band sublim tell someth wrong c...
54826     interest fact huge fuck tool suck enorm amount...
35554     point think whole articl dedic hillari smear c...
145216                    fuck stupid ignor love metal band
Name: comment_text, dtype: object

In [9]:
X_raw = texts['clean']
X_train, X_test, y_train, y_test = train_test_split(X_raw, labels, test_size=0.2, random_state=42,shuffle = True)

# Passage du texte dans un format structuré

In [10]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',                 
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words = stopwords_,
    sublinear_tf=True)

word_vectorizer.fit(X_train)
X_train_transformed = word_vectorizer.transform(X_train)
X_test_transformed = word_vectorizer.transform(X_test)


# Entraînement du modèle baseline

In [11]:
model = sklearn.ensemble.RandomForestClassifier(
     n_estimators=100,
     criterion='gini',
     max_depth=None,
     min_samples_split=2,
     min_samples_leaf=1,
     min_weight_fraction_leaf=0.0,
     max_features='auto',
     max_leaf_nodes=None,
     min_impurity_decrease=0.0,
     bootstrap=True,
     oob_score=False,
     n_jobs=None,
     random_state=None,
     verbose=0,
     warm_start=False,
     class_weight=None,
     ccp_alpha=0.0,
     max_samples=None,)
model.fit(X_train_transformed[:1000],y_train[:1000])

RandomForestClassifier()

In [12]:
model.score(X_test_transformed[:1000],y_test[:1000])

0.892

# Itération de la modélisation 

In [13]:
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization

In [14]:
model = Sequential()
model.add(LSTM(64))
model.add(Dense(32,activation='relu'))
model.add(Dense(6,activation='sigmoid'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_transformed = np.array(X_train_transformed)
y_train = np.array(y_train)
X_test_transformed = np.array(X_test_transformed)
y_test = np.array(y_test)
history = model.fit(X_train_transformed[:1000],y_train[:1000],epochs=20,validation_data=(X_test_transformed[:1000],y_test[:1000]),batch_size=256,verbose=1)

IndexError: ignored

In [ ]:

X_train_transformed = np.array(X_train_transformed)
